# Reference

https://medium.com/@maxpastor/how-to-set-up-an-autoscaling-kubernetes-cluster-on-amazon-eks-with-rancher-932263b532c6

# Kubernetes & Rancher
Kubernetes is a great tool that allows you to deploy and scale services in a cluster of servers. It’s great, but what a mess to set up! You have to write YAML files that have more lines than a zebra with a CLI that’s cool but not that visual. When you manage a complex infrastructure it’s nice to have visual feedback on what’s happening at the moment.
And that’s why we have Rancher. An easy to set up dashboard that allows you to create and manage your Kubernetes clusters the way you want to. Visually or with the CLI.
I won’t go into the details of installing Rancher on a machine. If you are here you might already have an instance of Rancher running. If not, just go on their website and follow the instructions. It’s super simple!

https://rancher.com/

## AWS EKS
Now when you have a Kubernetes cluster running, you have to scale it to meet the demand. But, if you’re Uber or Deliveroo, you might need 100 servers running the cluster at 9 pm but at 2 am …. You won’t need these resources anymore. So what do you do? Do you struggle to change the size of the cluster every day by clicking on buttons or do you use a tool that does that on its own?
And that’s why Amazon EKS is a good choice. Because this service allows you to create autoscaling policies that will grow the size of your infrastructure when you have a peak of traffic and then shrink it when you don’t need that much computing power anymore.
It saves you time, money, the trouble of not having sized the cluster properly and it’s kind of a green approach since you’re not wasting power anymore.
Thanks for the explanation man, but how do I mount this thing? — Thomas, the passionate CTO without much patience
Well, let’s go then. Let’s set this beast up!

## Creating the IAM policy and user

It’s not a surprise if you’re familiar with AWS, when you interact with AWS via the API, you have to create an IAM policy and the user that comes with it. This allows you to secure your integration by allowing the client to do only what you want it to do.
I invite you to go to your AWS Console to create a new policy and click on JSON.
This is what you want to paste in it:


```
{
 “Version”: “2012–10–17”,
 “Statement”: [
 {
 “Sid”: “VisualEditor0”,
 “Effect”: “Allow”,
 “Action”: [
 “elasticloadbalancing:ModifyListener”,
 “elasticloadbalancing:DetachLoadBalancerFromSubnets”,
 “elasticloadbalancing:ConfigureHealthCheck”,
 “elasticloadbalancing:RegisterTargets”,
 “autoscaling:DescribeLaunchConfigurations”,
 “autoscaling:*”,
 “elasticloadbalancing:DeleteLoadBalancer”,
 “elasticloadbalancing:DescribeLoadBalancers”,
 “elasticloadbalancing:CreateListener”,
 “elasticloadbalancing:DescribeListeners”,
 “autoscaling:DescribeAutoScalingGroups”,
 “ec2:CreateInternetGateway”,
 “kms:*”,
 “elasticloadbalancing:DescribeLoadBalancerPolicies”,
 “elasticloadbalancing:ApplySecurityGroupsToLoadBalancer”,
 “tag:*”,
 “elasticloadbalancing:DeregisterInstancesFromLoadBalancer”,
 “elasticloadbalancing:RegisterInstancesWithLoadBalancer”,
 “elasticloadbalancing:CreateLoadBalancerPolicy”,
 “elasticloadbalancing:SetLoadBalancerPoliciesForBackendServer”,
 “elasticloadbalancing:CreateLoadBalancer”,
 “elasticloadbalancing:AttachLoadBalancerToSubnets”,
 “autoscaling:DescribeTags”,
 “elasticloadbalancing:CreateTargetGroup”,
 “cloudformation:*”,
 “elasticloadbalancing:*”,
 “autoscaling-plans:*”,
 “elasticloadbalancing:DeleteTargetGroup”,
 “elasticloadbalancing:CreateLoadBalancerListeners”,
 “iam:*”,
 “elasticloadbalancing:DescribeLoadBalancerAttributes”,
 “elasticloadbalancing:SetLoadBalancerPoliciesOfListener”,
 “elasticloadbalancing:AddTags”,
 “elasticloadbalancing:DescribeTargetHealth”,
 “elasticloadbalancing:DescribeTargetGroups”,
 “elasticloadbalancing:DeleteLoadBalancerListeners”,
 “ec2:*”,
 “eks:*”,
 “elasticloadbalancing:ModifyLoadBalancerAttributes”,
 “elasticloadbalancing:ModifyTargetGroup”,
 “elasticloadbalancing:DeleteListener”
 ],
 “Resource”: “*”
 }
 ]
}

```

These permissions will allow our user to create all of the resources that we will need to launch an EKS cluster and to create a classic Kubernetes cluster from Rancher without EKS (the old way).
Now that you’ve named and described your policy it’s time to create a user.


![](https://miro.medium.com/max/1400/1*H_gCJzp9_ZYkr2fv8Gk8pQ.png)

Name it and specify that you want a Programmatic Access.
On the next step, click on “Attach existing policies directly” and search for the policy we’ve just created. Then add tags.
It’s not necessary to add tags, but people from AWS will advise you to put some on so you can find your resources more easily and have a better visualization over your costs.
You can now create your user.
Now the console shows you your keys. Download the CSV and keep it in a safe place.
Congratulations. We now have everything we need to create the cluster. So let’s jump on Rancher and launch this thing!

## Creating the EKS cluster via Rancher
So you are now on your Rancher dashboard. Click on Add cluster. And you should land on something that looks like this.

![](https://miro.medium.com/max/1400/1*pgHGx-Zpa8gRflb1QEIMnw.png)

So obviously you will choose Amazon EKS and from there it’s easy. Choose your region, pass the credentials that you downloaded earlier, select the default options each time until you get to the node section.

![](https://miro.medium.com/max/1400/1*U6OzQUcx_iDd_JWQg9UHDg.png)

There you will want to customize things. Select the instance that suits your needs, select the minimum cluster size (when your cluster will shrink it will not go below this number), same thing with the maximum cluster size. Then, select how much GB you want for each machine. Click create and you’re good to go!
At that point, you will have to wait for 10 to 20 minutes while everything magically rises from the ground.

## Configuring your cluster
This might be why you came here and this is why I wrote this. It took me a lot of time to figure out how to set everything up properly.
How do you route your traffic to a service? How do you automatically sync your route53 DNS records with your Application Load Balancer? How do you launch the autoscaling part of the infrastructure?
Well, this part will guide you through all of this so that it works and you know how to customize things later.

## Creating your first service
To start we are going to create a simple HelloWorld service. It consists of a simple express app that will return a basic message when we hit a specific URL.
If you have such a service containerized go ahead and use it, if not I’ve set one up for you.
I invite you to go to your cluster’s default namespace and click on deploy.

![](![image.png](attachment:image.png))

![](https://miro.medium.com/max/1400/1*gQ6xriAamVx0zClYOkq3Qg.png)

![](https://miro.medium.com/max/1400/1*O9RXPzlOZ84S23b9iN7GeA.png)

Now you want to use tactill/helloworld image, map the port 8080 and add an environment variable named PORT with the value 8080 so that the script knows which port to map.

Scroll down and click “Launch”

## Routing with alb ingress controller
Let’s dive into the more technical part of things.
First, you will have to create a new policy in your IAM console. This policy will allow the user that you created to manage resources related to the Application Loadbalancer. Same procedure as the last one, you can find the JSON here.

When the policy is created, just add it to your user. You then have to download this aws-alb-ingress-controller.yaml file and change the values to match your config. You will have to uncomment and change the cluster name, the aws credentials, the region, and the vpc.
To find the VPC of your cluster you just have to go to your AWS console, and take a look at your EKS cluster. It looks like this:

![](https://miro.medium.com/max/1400/1*nDRV1xWnKHSRXGiks5DXtQ.png)

When your file is ready, go back to your Rancher cluster. Click on your cluster name and you will find this page.

![](https://miro.medium.com/max/1400/1*IRYWIbVIP7uJcrUCptIHsw.png)

Click on “Launch kubectl” so we have access to the Kubernetes CLI.
In the terminal, you will have to download your files. Make sure to do this securely or that it’s only temporarily accessible because it contains sensitive credentials in them. When your files are on the cluster you will be able to launch the Kubernetes controller. It looks like this:

```
curl -sS https://xxxx.xx/aws-alb-ingress-controller.yaml > controller.yaml
curl -sS https://raw.githubusercontent.com/kubernetes-sigs/aws-alb-ingress-controller/master/docs/examples/rbac-role.yaml > rbac-role.yaml
kubectl apply -f rbac-role.yaml
kubectl apply -f controller.yaml
```

## Automatic Route 53 synchronization
We are going to add the last dependency and this one will help us with DNS level routing. It means that when we will be creating a route in rancher, it will create, update or delete DNS records so we won’t have to go to the AWS console to do it ourselves.
First, you will need to add a new IAM policy to your user. Here’s the JSON.

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "route53:ChangeResourceRecordSets"
            ],
            "Resource": [
                "arn:aws:route53:::hostedzone/*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "route53:ListHostedZones",
                "route53:ListResourceRecordSets"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}
```


Then, you will need to customize this file. The things you need to change are the domain name you’re targetting and the policy. By default, it’s set to upsert-only which means that it will only create and update records. Personally, I set it to sync so it can delete records too. Yet, it also means that you will have to be careful when playing with your routes.
Let’s go back to the Rancher dashboard in Default and click “import yaml”.
Upload your file and click on import.
Great! We’re close to the end. The last thing we need to do is to actually create the ingress for the actual routing!

## Creating the LoadBalancer
Go into your default namespace and click on Loadbalancing.

![](https://miro.medium.com/max/1400/1*OAnb8QozvJPAnlOuVkNcLA.png)

Click on add ingress and get ready for the final configuration.

![](https://miro.medium.com/max/1400/1*I6BnNOJtgDMMOD9mPbMxzw.png)

You will have to specify a hostname (that our DNS manager will create in Route53) and a service you want to route to.
Then, scroll a bit and add a few annotations. This will let our ingress-controller know that we want to route this thing and create an ALB.

![](https://miro.medium.com/max/1400/1*2IwwCvm7TOhPOzS1nJXxsQ.png)

You can find your subnets in your EKS cluster page in your AWS console. The same page where we found the VPC name.
Click on save and you’re ready to go!
Just wait a few minutes for everything to be created and propagated and then go to ekstutorial.yourdomain.com/helloworld.

## Autoscaling the cluster
The cluster will not autoscale by default. You have to create the policy in your AWS console. If you look in your Rancher dashboard you will see that by default you have 3 nodes running. It’s a wait of resources for a tutorial. So let’s set up the autoscaling policy so that it shrinks to 1 node and adapts to the demand.
Go to https://console.aws.amazon.com/awsautoscaling/home?region=us-east-1#dashboard and add a scaling plan.
Choose EC2 scaling group and select yours.

![](https://miro.medium.com/max/1400/1*9axqPMkHdsfeJdxKm1UG6w.png)

Configure it for your needs.

![](https://miro.medium.com/max/1400/1*srdc5x7dPjNEmVTx7DF8GA.png)

And just create it!
If you go back to your nodes on Rancher you will see them shrink.
You now have an autoscaling eks cluster managed with Rancher
# Conclusion
So this was a journey! I tried to make it as easy as possible while keeping a detailed technical approach. I hope that you made it here and that your cluster works great.
If you have any comments or suggestions feel free to ask them, I’ll be glad to help you and adjust this article if needed!
See you soon for another one of these!